In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd


df = pd.read_csv("../data/tass.csv")

df

,id,text,lang,polarity,split
0,768512386269638656,@morbosaborealis jajajaja... eso es verdad... ...,es,NEG,train
1,768529956162924544,@Adriansoler espero y deseo que el interior te...,es,NEU,train
2,768557093955698688,"comprendo que te molen mis tattoos, pero no te...",es,NEU,train
3,770616744192929792,"Mi última partida jugada, con Sona support. La...",es,POS,train
4,769959690092642304,Tranquilos que con el.dinero de Camacho seguro...,es,POS,train
...,...,...,...,...,...
14504,756565904037670912,Un saludo para don Resentidillo muñoz.! Que si...,PE,POS,test
14505,757602234041524224,"Sí, es que se nota que te importa... - Buen in...",PE,NEU,test
14506,786072417307328512,"@Isoor_ , @iChibolo , @FloroPeruano , @latiach...",PE,NEG,test
14507,794135682876919808,@bcrpoficial Cash Miguel @canalN_ Manu Ramirez...,PE,NEU,test


In [3]:
from pysentimiento.tass import label2id
df["label"] = df["polarity"].apply(lambda x: label2id[x])

df["split"].value_counts()

test     7264
train    4802
dev      2443
Name: split, dtype: int64

In [1]:
from transformers import AutoTokenizer
from finetune_vs_scratch.preprocessing import special_tokens

model_name = "dccuchile/bert-base-spanish-wwm-uncased"

tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")
tokenizer.model_max_length = 128
tokenizer.add_tokens(special_tokens)



4

In [7]:
from glob import glob

tweet_files = glob("../data/filtered_tweets/*.txt")

train_files = tweet_files[:-1]
test_files = tweet_files[-1:]
len(train_files), len(test_files)

(99, 1)

In [8]:
from datasets import load_dataset, Features, Value


features = Features({
    'text': Value('string'),
})

train_dataset, test_dataset = load_dataset(
    "text", data_files={"train": train_files, "test": test_files}, split=["train", "test"], features=features
)

Using custom data configuration default-3b674591494fed6a


0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset text downloaded and prepared to /home/jmperez/.cache/huggingface/datasets/text/default-3b674591494fed6a/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5. Subsequent calls will reuse this data.


In [11]:
train_dataset.save_to_disk("../data/arrow/train")

In [12]:
test_dataset.save_to_disk("../data/arrow/test")

## Reloading

Mirar este issue: https://github.com/huggingface/transformers/issues/10204

In [1]:
import datasets
from datasets import load_dataset, Features, Value, load_from_disk

train_dataset = load_from_disk("../data/arrow/tokenized_train")
test_dataset = load_from_disk("../data/arrow/tokenized_test")

train_dataset, test_dataset

(Dataset({
     features: ['attention_mask', 'input_ids', 'special_tokens_mask', 'text', 'token_type_ids'],
     num_rows: 25600000
 }),
 Dataset({
     features: ['attention_mask', 'input_ids', 'special_tokens_mask', 'text', 'token_type_ids'],
     num_rows: 5215789
 }))

In [4]:
import torch
from transformers import AutoTokenizer
from finetune_vs_scratch.preprocessing import special_tokens
from transformers import BertForMaskedLM, BertTokenizerFast
model_name = 'dccuchile/bert-base-spanish-wwm-cased'

tokenizer = BertTokenizerFast.from_pretrained(model_name)
tokenizer.model_max_length = 128
tokenizer.add_tokens(special_tokens)


4

In [5]:
example = train_dataset[10900]

example["text"]

'Taylor Swift - End Game ft. Ed Sheeran, Future URL via @usuario'

In [6]:
tokenizer.decode(example["input_ids"])

'[CLS] Taylor Swift - End Game ft. Ed Sheeran, Future URL via @usuario [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [7]:
tokenizer.save_pretrained("../data/beto-tokenizer")

('../data/beto-tokenizer/tokenizer_config.json',
 '../data/beto-tokenizer/special_tokens_map.json',
 '../data/beto-tokenizer/vocab.txt',
 '../data/beto-tokenizer/added_tokens.json',
 '../data/beto-tokenizer/tokenizer.json')

In [8]:
tweets = train_dataset[:2048*10]["text"]

In [11]:
from datasets import Dataset

dataset = Dataset.from_dict({"text": tweets})

In [13]:

def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, return_special_tokens_mask=True)
batch_size = 2048

dataset = dataset.map(tokenize, batch_size=batch_size, batched=True)

  0%|          | 0/10 [00:00<?, ?ba/s]

In [14]:

dataset.save_to_disk("../data/arrow/small_dataset")

In [15]:
!pwd

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/jmperez/projects/finetune_vs_scratch/notebooks


In [27]:

batch_size = 2048
steps = 12_500

needed_tweets = batch_size * steps

needed_tweets

25600000

In [22]:
import random

random.seed(2021)

idx = random.sample(list(range(len(train_dataset))), 10*batch_size)

train_dataset = train_dataset.select(idx)

In [23]:

len(train_dataset)

20480

In [24]:
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, return_special_tokens_mask=True)



train_dataset = train_dataset.map(tokenize, batch_size=batch_size, batched=True)

  0%|          | 0/10 [00:00<?, ?ba/s]

In [25]:
train_dataset.save_to_disk("prueba")